## Ejercicio 1: Procesamiento de datos

### By:
Auberth Eduardo Hurtado

### Date:
2025-01-17

### Description:

presentation of methodological proposal for data analysis.


## 📚 Libraries used

In [1]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib

# Switch to an interactive backend
matplotlib.use('TkAgg')  #Optional

## 💾 Tareas
### 1. Propón y describe brevemente una estrategia para manejar los datos faltantes, considerando tanto la imputación como la eliminación.

#### Estrategia para el Manejo de datos faltantes
1. Identificación de Datos Faltantes:
- Haría un EDA muy rápido utilizando algun framework como pandas profiling para determinar las estadísticas descriptivas generales, el objetivo es tener una vista panorámica de qué está pasando en los datos
- Luego empezaría Utilizando métodos como isnull() o isna() para identifica en detalle campos con datos vacíos o faltantes.

2. Imputación de Datos:
Aqui separaría el dataset en diferentes condiciones de los tipos de datos
- **Campos Numéricos:** Imputar valores numéricos faltantes utilizando la media, mediana o moda de la columna, otro camino es utilizando algoritmos de imputación basados en vecindades o segmentos. Ejemplo: Para precios, se puede imputar utilizando el promedio de productos similares.
- **Campos Categóricos:** Imputar valores categóricos faltantes utilizando la moda de la columna o el valor más frecuente, se puede revisar la distribución de la variable para identificar la posibilidad de crear nuevas categorías.
- **Campos Temporales:** Imputar valores temporales faltantes utilizando interpolación o el valor más cercano, además dependiendo de los datos se puede considerar análisis en ventanas de tiempo o móvil.

3. Eliminación de Datos Faltantes:
- Si una fila tiene demasiados valores faltantes, se puede considerar eliminarla.
- Si una columna tiene demasiados valores faltantes y no es crucial para el análisis, se puede considerar eliminarla.

**Nota:** considero importante mencionar que existen otros casos en los cuales se deben tener el cuenta la validez del formato de los campos o en el formato esperado, existen estrategias que utilizan framewors como __great_expectations__ que permiten realizar distintas validaciones y que quedarían automatizadas en el modelo en producción

### Load data

In [ ]:
raw_sales_data = pd.read_csv(r'../data/raw/raw_sales_data.csv')
raw_sales_data.head()

## 👷 2. Limpia y procesa los datos
- Identifica valores inválidos o fuera de rango. 
- Realiza imputaciones dinámicas según patrones detectados (por ejemplo, completar price basado en el promedio de productos similares). 

In [ ]:
profile = ProfileReport(raw_sales_data, title="Profiling Report")
profile.to_notebook_iframe()

- Identifica valores inválidos o fuera de rango. 


In [4]:
noise_options = ['NULL', -9999, "-9999", np.nan]

# Replace values 'NULL', -9999, or np.nan with np.nan
raw_sales_data.replace(noise_options, np.nan, inplace=True)

# Identify missing values
missing_values = raw_sales_data.isnull().sum()
#missing_values

- Realiza imputaciones dinámicas según patrones detectados (por ejemplo, completar price basado en el promedio de productos similares). 

In [5]:
# Visualize the pattern of missing values
plt.figure(figsize=(12, 8))
sns.heatmap(raw_sales_data.isnull(), cbar=False, cmap='viridis')
plt.savefig('../docs/missing_values_heatmap.png')
#plt.show()

El anterior resultado muestra que el patrón de valores perdidos es totalmente aleatorio, sin embargo para efectos de la prueba vamos a asumir que la ubicación geográfica (región) recoge las variaciones interna de los clientes.

Adicionalmente, vamos a asumir que ya se agotaron todas las instacias y es imposible recuperar datos de cliente (customer_id) y región (region).

In [ ]:
# Remove rows with null values in 'customer_id' and 'region'
raw_sales_data.dropna(subset=['customer_id', 'region', 'order_date'], inplace=True)

# [Optional] Drop rows with too many missing values
# raw_sales_data.dropna(thresh=len(raw_sales_data.columns) - 3, inplace=True)

# Impute other fields dynamically
raw_sales_data['product_id'] = raw_sales_data.groupby('region')['product_id'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.median()))
raw_sales_data['quantity'] = raw_sales_data.groupby('region')['quantity'].transform(lambda x: x.fillna(int(x.median())))
raw_sales_data['price'] = raw_sales_data.groupby('region')['price'].transform(lambda x: x.fillna(x.mean()))
raw_sales_data['discount'] = raw_sales_data.groupby('region')['discount'].transform(lambda x: x.fillna(x.median()))
raw_sales_data['shipping_priority'] = raw_sales_data.groupby('region')['shipping_priority'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x.median()))
raw_sales_data.head()

In [ ]:
# Validation
raw_sales_data.isnull().sum()

## 👷 3. Calcula: 
- Ingreso total por cliente considerando descuentos. 
- Producto más vendido por región y ingreso total generado por cada región. 
- Distribución de prioridad de envío por región. 

In [ ]:
# Calculate total income per customer considering discounts
raw_sales_data['total_income'] = raw_sales_data['quantity'] * raw_sales_data['price'] * (1 - raw_sales_data['discount'])
total_income_per_client = raw_sales_data.groupby('customer_id')['total_income'].sum().reset_index()
total_income_per_client.head()


- Producto más vendido por región y ingreso total generado por cada región. 

In [ ]:
# Most sold product by region
most_sold_product = raw_sales_data.groupby('region')['product_id'].agg(lambda x: x.value_counts().index[0])
most_sold_product.head()

In [ ]:
# Total income generated by each region
total_income_by_region = raw_sales_data.groupby('region')['total_income'].sum()
total_income_by_region.head()

- Distribución de prioridad de envío por región. 

In [ ]:
# Distribution of shipping priority by region
shipping_priority_distribution = raw_sales_data.groupby('region')['shipping_priority'].value_counts(normalize=True)
shipping_priority_distribution.head()

## 📊 4. Guarda el dataset limpio y procesado en cleaned_sales_data.csv


In [12]:
# Save the cleaned and processed dataset
raw_sales_data.to_csv(r'../data/processed/cleaned_sales_data.csv', index=False)

## 💡 Caveats

In [ ]:
print(f'Data cleaning and processing completed successfully with {raw_sales_data.shape[0]} rows')